In [1]:
from gensim import corpora,models,similarities,utils
import jieba
import jieba.posseg as pseg
import re

In [2]:
jieba.load_userdict("/Users/jack/Desktop/LDA/dict.txt.big")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/z0/0vtl52ys4js7_0rnmqt9bdfr0000gt/T/jieba.cache
Loading model cost 1.272 seconds.
Prefix dict has been built succesfully.


In [3]:
all_news = ['好友的母親出門倒垃圾，一輛急駛摩托車猛然撞擊，就此倒地不起。這位伯母原本有心臟宿疾，家裡隨時準備著氧氣筒。然而萬萬沒有料到，她是用這種方式離開。子女完全不能接受，哭著說：「媽媽一句交代都沒就走了！」他們以為，媽媽即使心臟病發作，也總還有時間跟他們說說話，交代幾句，怎麼可以一聲不響就走呢？其實，他們忘了，媽媽每天都在交代。就跟天下的母親一樣，無非是「注意身體，小心著涼」、「不要太累，少熬夜，少喝酒」、「好好念書，別整天貪玩」......只不過我們聽得太多，聽得我們煩膩、麻木。直到母親閉口的那刻，我們才發現，還有很多話來不及聽、來不及問、來不及跟媽媽說。',
           '一位母親，因為女兒愛上一個她不喜歡的男人，母女僵持不下，大吵一架後，女兒乾脆離家。母親又氣又傷心，女兒自小失怙，是她母兼父職辛苦養大。好不容易出落得亭亭玉立，水仙花兒似的，誰知大學尚未畢業，就急著想嫁，偏又是位大她?多歲的離婚男人。母親好言相勸，惡言恫嚇，女兒不動如山。所有的愛變成恨。她恨女兒絕情，為愛盲目。許多前塵往事一一湧上頭。女兒小時乖巧可愛，老愛膩在她身邊嘰嘰咕咕像小雞啄米似的講悄悄話。童言童語，煞是有趣。「媽媽，你絕不能先老，一定要等我長大了一起老！」上中學的女兒也依然貼心懂事，母女倆像朋友一般分享彼此的心事。偶爾問起女兒擇偶的條件，女兒總撒嬌地說：「我才不嫁，我要陪媽媽一輩子，陪到你老得走不動，我就幫你推輪椅！」這些話言猶在耳，女兒怎麼全忘了呢？為了一個不相干的男人，罔顧二年母女情份，實在叫她難以承受。那天，女兒打電話回來說：「媽媽，我要結婚了，希望你來參加婚禮，給我一點祝福！」她餘怒未消，憤而掛上電話。這一掛就是生死永隔。女兒女婿在蜜月途中車禍喪生。殯儀館內，她抱著女兒的遺體放聲大哭：「我好自私啊！我連最後的祝福都不肯給你！」',
           '第一天上學的小朋友哭的很可憐，老師問他原因，他說：「我不喜歡學校，可是以後我得天天來這你，一直到15歲。」老師安慰她道：「我比你更可憐，我得天天來這裡，一直到60歲呢！」',
           '這天，老師發考卷……「廖淑芬，６０分！你呀，不是名字叫起來像６０分就 可以考６０分啊？！」老師不悅地說…「伍淑芬，５０分！你呀！比廖淑芬還不如！」老師依 舊忿忿然地說著…接著，老師以更生氣的語調發著下一張考卷：柯淑芬（台語），１０分！喔，你也一樣！名字叫起 來像1０分你就給我考１０分？！……」這時……「啊！我完了……」柯淑芬的妹妹心中開始暗叫不妙……老師嘆口氣，很無奈的發著第三張：「不是我在說你們 這對姊妹實在是太不像話了……」「柯玲芬（台語）……０分！」老師搖搖頭道：「你們兩姐姐應該向你們哥哥好好看 齊，他總是滿分，你們倆不要老是考那樣的成績。」柯淑芬和柯玲芬兩姐妹心裡暗罵：「都嘛是爸媽把哥哥的名字取做…………柯吉霸！！！',
           '美國在當地6日時間舉行期中選舉，包含36州州長、參議院35席以及眾議院435席都在此次選舉中改選。依據美國選舉制度，美國國會每兩年改選一次，每次改選三分之一的參議院席次及全部眾議院席次。在上一屆2016年總統暨國會大選中，代表共和黨的川普贏得了總統選舉，而在參、眾議院席次的部分，共和黨也取得了52席參議院席次，以及235席眾議院席次的過半成績。在參議院部分，由於此次期中選舉改選的35席中有24席皆為民主黨所有，因此改選結果預估對共和黨影響並不大。而在眾議院部分，上一屆共和黨在眾議院取得了過半席次，因此此次民主黨若想在眾議院成為多數黨，除了必須保住目前席位外，還需額外取得25席眾議院席次。外界普遍將這次期中選舉，視為是對川普就任兩年以來的政績評分，而此次選舉結果也將影響川普未來兩年剩餘任期的施政難易度。以下聯合新聞網為您即時更新美國期中選舉開票結果。',
           '一對父母帶著6個月大的女兒去打預防針，因為難忍疼痛，女嬰嚎啕大哭，父親抱起來安撫，沒想到女嬰竟然發出疑似國罵「三字經」的聲音，讓爸媽聽了忍不住笑了出來，影片放到網路上讓網友笑翻，直呼「太可愛了」。這對父母在臉書「爆廢公社」PO出一段影片，他們帶著女兒去打預防針，爸爸抱著她讓護理人員打針，媽媽則在一旁錄影。打針時女嬰痛得大哭，媽媽邊錄影邊逗弄著小玩具安撫她的情緒，打完針之後，再由爸爸抱起來，但女嬰仍哭得悽慘，沒想到她突然疑似國罵「X恁娘」的聲音，讓爸媽頓時爆笑的對著女嬰反問說「妳剛剛說什麼？哈囉，妳再說一次，妳剛剛說什麼？」網友看完影片後大笑「好清楚」、「哈哈，這是誰教的」、「我重複看了五遍笑了五遍」、「不錯喔，這孩子有前途」、「寶寶心裡哭，寶寶直接罵出來」、「發現說錯話馬上閉嘴」、「笑死我了，罵完還瞪媽媽」、「哈哈哈哈…忘記喝孟婆湯了啦」。',
           '花旗銀行因為1名信用卡持卡人，惡意利用「預繳卡費」功能，1個多月內被詐騙達6300萬元，遭金管會處罰。據了解，本案就是今年6月被警方破獲的桃園「烤雞王」老闆陳星文狂刷信用卡詐欺案。相關人士透露，本案起因的確因為花旗的系統「有漏洞（bug）」，但漏洞不是出在花旗信用卡的系統，而是花旗銀行系統有bug。據了解，這名陳姓持卡人，某次打算透過ATM預繳卡費到花旗信用卡帳上時，結果轉帳失敗，但因為銀行系統有bug，這個轉帳失敗的訊息，卻變成成功。也就是說，假設持卡人本來想轉個1萬元預繳卡費，明明就轉帳失敗，「1毛錢都沒轉進去」，但因為系統有bug，當事人的信用卡額度在轉帳後卻虛增1萬。據了解，這名持卡人發現他轉帳失敗後，「信用卡額度卻擴增」，見獵心喜、屢次嘗試，每次都轉帳不成功，但卡片信用額度卻一直不停擴增，一直膨脹到6000多萬元。這名陳姓持卡人就靠因為系統出現bug，導致虛增的信用卡額度大刷特刷，大買高價精品、名表、珠寶、名牌包及禮券再以低於市價轉手。據了解，銀行後來是發現陳姓持卡人的信用額度不斷擴增到離譜境界，但根本就沒錢進來信用卡帳上才察覺有異，趕緊在第一時間修復系統並立即報警。警方也趁當事人出境前，逮捕歸案。花旗銀行在案發後已發布聲明，本案是花旗主動發現，在第一時間呈報主管機關與檢警單位，經清查所有相關交易紀錄與文件，確認此案為蓄意詐騙花旗的單一個案，沒有其他客戶權益受到影響。由於本案已進入司法程序，不便再對外說明案件細節。但花旗將持續強化各項風險控管機制，確保客戶權益，懇請客戶放心。']

In [4]:
#移除括號內的內容
remover1=re.compile(r"\（.*?\）")
remover2=re.compile(r"\[.*?\]")
remover3=re.compile(r"\〔.*?\〕")
remover4=re.compile(r"\(.*?\)")

for i in range(len(all_news)):
        all_news[i]=remover1.sub("",all_news[i])
        all_news[i]=remover2.sub("",all_news[i])
        all_news[i]=remover3.sub("",all_news[i])
        all_news[i]=remover4.sub("",all_news[i])

In [5]:
all_news[1]

'一位母親，因為女兒愛上一個她不喜歡的男人，母女僵持不下，大吵一架後，女兒乾脆離家。母親又氣又傷心，女兒自小失怙，是她母兼父職辛苦養大。好不容易出落得亭亭玉立，水仙花兒似的，誰知大學尚未畢業，就急著想嫁，偏又是位大她?多歲的離婚男人。母親好言相勸，惡言恫嚇，女兒不動如山。所有的愛變成恨。她恨女兒絕情，為愛盲目。許多前塵往事一一湧上頭。女兒小時乖巧可愛，老愛膩在她身邊嘰嘰咕咕像小雞啄米似的講悄悄話。童言童語，煞是有趣。「媽媽，你絕不能先老，一定要等我長大了一起老！」上中學的女兒也依然貼心懂事，母女倆像朋友一般分享彼此的心事。偶爾問起女兒擇偶的條件，女兒總撒嬌地說：「我才不嫁，我要陪媽媽一輩子，陪到你老得走不動，我就幫你推輪椅！」這些話言猶在耳，女兒怎麼全忘了呢？為了一個不相干的男人，罔顧二年母女情份，實在叫她難以承受。那天，女兒打電話回來說：「媽媽，我要結婚了，希望你來參加婚禮，給我一點祝福！」她餘怒未消，憤而掛上電話。這一掛就是生死永隔。女兒女婿在蜜月途中車禍喪生。殯儀館內，她抱著女兒的遺體放聲大哭：「我好自私啊！我連最後的祝福都不肯給你！」'

In [6]:
with open('/Users/jack/Desktop/LDA/stopword.txt', encoding='utf-8') as f:
    stop_word_list = f.readlines()
    
stop_word_list = [x.strip() for x in stop_word_list]

In [7]:
texts = []
for i in all_news:
    tokens = jieba.cut(i,cut_all=False,HMM=True)
    stopped_tokens = [i for i in tokens if not i in stop_word_list]
    texts.append(stopped_tokens)

In [8]:
#texts

In [9]:
# print(dictionary)

In [10]:
#dictionary = corpora.Dictionary(texts)
dictionary = corpora.Dictionary.load('/Users/jack/Desktop/LDA/allnews_dictionary.dict')

In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [12]:
#tfidf_model = models.tfidfmodel.TfidfModel.load("/Users/wangguoquan/Desktop/LDA/tfidf_model.tfidf")

In [13]:
tfidfModel = models.tfidfmodel.TfidfModel.load("/Users/jack/Desktop/LDA/tfidf_model.tfidf")

In [14]:
tfidfVectors = tfidfModel[corpus]

In [15]:
#indexTfidf = similarities.MatrixSimilarity(tfidfVectors)

In [16]:
ldamodel = models.ldamodel.LdaModel.load('/Users/jack/Desktop/LDA/allnews_LDA_model.lda')

In [17]:
#corpus_lda = ldamodel[tfidfVectors]

In [18]:
#indexLDA = similarities.MatrixSimilarity(corpus_lda)

In [19]:
#整理一行
indexLDA = similarities.MatrixSimilarity(ldamodel[tfidfVectors])

/Users/jack/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [20]:
doc = "想 把 你 寫成 一首歌 想養 一隻 貓 想要 回到 每個 場景 撥慢 每 隻 錶 我們 在 小孩 和 大人 的 轉角 蓋 一座 城堡 我們 好好 好 到 瘋 掉 像 找回 失散多年 雙胞 生命 再長 不過 煙火 落下 了 眼角 世界 再大 不過 你 我 凝視 的 微笑 在 所有 流逝 風景 與 人群 中 你 對 我 最好 一切 好好 是否 太好 沒有 人 知道 你 和 我 背著 空空 的 書包 逃出 名為 日常 的 監牢 忘 了 要 長大 忘 了 要 變老 忘 了 時間 有腳 最 安靜 的 時刻 回憶 總是 最 喧囂 最 喧囂 的 狂歡 寂寞 包圍 著 孤島 還以 為 馴服 想念 能 陪伴 我 像 一隻 家貓 它 就 窩 在 沙發 一角 卻 不肯 睡著 你 和 我 曾 有 滿滿的 羽毛 跳 著名 為 青春 的 舞蹈 不 知道 未來 不 知道 煩惱 不知 那些 日子 會 是 那麼 少 時間 的 電影 結局 才 知道 原來 大人 已 沒有 童謠 最後 的 叮嚀 最後 的 擁抱 我們 紅著 眼笑 我們 都 要 把 自己 照顧 好 好 到 遺憾 無法 打擾 好好 的 生活 好好 的 變老 好好 假裝 我 已經 把 你 忘掉 "

In [21]:
#vec_bow = dictionary.doc2bow(doc.split())

In [22]:
#vec_lda = ldamodel[vec_bow]

In [23]:
#print(vec_lda)

In [24]:
#sims = indexLDA[vec_lda]

In [25]:
#整理為一行
sims = indexLDA[ldamodel[dictionary.doc2bow(doc.split())]]

In [26]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [27]:
print(sims)

[(5, 0.86630785), (0, 0.86465776), (3, 0.86237764), (6, 0.771787), (1, 0.7679315), (2, 0.73584634), (4, 0.51751316)]


In [ ]:
#參考資料
#https://github.com/youngmihuang/lyrics_application/blob/master/lyrics_2.ipynb
#https://zhuanlan.zhihu.com/p/21364664